In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
poster = pd.read_csv('/content/drive/MyDrive/기학팀플/포스터/poster_with_cluster.csv')
data = pd.read_csv('/content/drive/MyDrive/기학팀플/movie.csv')

In [2]:
# === STEP 1: 데이터 준비 ===
# poster와 data를 movie_title을 기준으로 병합
data = data.merge(poster[['movie_title', 'cluster']], on='movie_title', how='left')
data

,critic_name,top_critic,review_type,review_content,movie_title,authors,actors,original_release_date,runtime,tomatometer_rating,...,directors_category,movie_info_keyword,author_id,actor_id,actors_split,actor_ids,movie_id,authors_split,predicted_emotion,cluster
0,Ben McEachen,0,1,Whether audiences will get behind The Lightnin...,Percy Jackson & the Olympians: The Lightning T...,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,0.49,...,1,life war mother son gets,0,0,"['Logan Lerman', 'Brandon T. Jackson', 'Alexan...","[76058, 130933, 53933, 107930, 33787, 182302, ...",9737,"['Craig Titley', 'Chris Columbus', 'Rick Riord...",worry,3
1,Nick Schager,0,0,Harry Potter knockoffs don't come more transpa...,Percy Jackson & the Olympians: The Lightning T...,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,0.49,...,1,life war mother son gets,0,0,"['Logan Lerman', 'Brandon T. Jackson', 'Alexan...","[76058, 130933, 53933, 107930, 33787, 182302, ...",9737,"['Craig Titley', 'Chris Columbus', 'Rick Riord...",neutral,3
2,Bill Goodykoontz,1,1,"Percy Jackson isn't a great movie, but it's a ...",Percy Jackson & the Olympians: The Lightning T...,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,0.49,...,1,life war mother son gets,0,0,"['Logan Lerman', 'Brandon T. Jackson', 'Alexan...","[76058, 130933, 53933, 107930, 33787, 182302, ...",9737,"['Craig Titley', 'Chris Columbus', 'Rick Riord...",fun,3
3,Jordan Hoffman,0,1,"Fun, brisk and imaginative",Percy Jackson & the Olympians: The Lightning T...,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,0.49,...,1,life war mother son gets,0,0,"['Logan Lerman', 'Brandon T. Jackson', 'Alexan...","[76058, 130933, 53933, 107930, 33787, 182302, ...",9737,"['Craig Titley', 'Chris Columbus', 'Rick Riord...",happiness,3
4,Jim Schembri,1,1,"Crammed with dragons, set-destroying fights an...",Percy Jackson & the Olympians: The Lightning T...,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,119.0,0.49,...,1,life war mother son gets,0,0,"['Logan Lerman', 'Brandon T. Jackson', 'Alexan...","[76058, 130933, 53933, 107930, 33787, 182302, ...",9737,"['Craig Titley', 'Chris Columbus', 'Rick Riord...",worry,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676576,Unknown,0,1,"This is that rarity in films - an all-action, ...",Zulu,"Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,135.0,0.96,...,4,help takes john daughter michael,12934,17251,"['Stanley Baker', 'Jack Hawkins', 'Ulla Jacobs...","[200217, 54469, 25690, 22298, 762, 144382, 180...",17005,"['Cy Endfield', 'John Prebble']",neutral,4
676577,Ken Hanke,0,1,"Seen today, it's not only a startling indictme...",Zulu Dawn,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,121.0,0.50,...,4,love make help young john,12935,17252,"['Burt Lancaster', ""Peter O'Toole"", 'Simon War...","[117760, 195542, 179449, 96539, 106184, 163464...",17006,"['Cy Endfield', 'Anthony Storey']",surprise,4
676578,Dennis Schwartz,0,1,A rousing visual spectacle that's a prequel of...,Zulu Dawn,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,121.0,0.50,...,4,love make help young john,12935,17252,"['Burt Lancaster', ""Peter O'Toole"", 'Simon War...","[117760, 195542, 179449, 96539, 106184, 163464...",17006,"['Cy Endfield', 'Anthony Storey']",happiness,4
676579,Christopher Lloyd,0,0,"A simple two-act story: Prelude to war, and th...",Zulu Dawn,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,121.0,0.50,...,4,love make help young john,12935,17252,"['Burt Lancaster', ""Peter O'Toole"", 'Si

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 필요한 열 정의
essential_cols = ['critic_name', 'movie_id', 'movie_title', 'review_score_cleaned', 'runtime', 'audience_rating',
                  'tomatometer_rating', 'predicted_emotion', 'movie_info_keyword', 'audience_status_Spilled',
                  'audience_status_Upright', 'cluster'] + [col for col in data.columns if col.startswith('genre_')]
data = data[essential_cols]

# 결측값 처리 및 정규화
data.loc[:, 'review_score_cleaned'] = data['review_score_cleaned'].fillna(data['review_score_cleaned'].mean())
data.loc[:, 'runtime'] = data['runtime'].fillna(data['runtime'].mean())
data.loc[:, 'audience_rating'] = data['audience_rating'].fillna(data['audience_rating'].mean())
data = data.dropna().reset_index(drop=True)
data['review_score_cleaned'] = (data['review_score_cleaned'] - data['review_score_cleaned'].min()) / \
                               (data['review_score_cleaned'].max() - data['review_score_cleaned'].min())

# predicted_emotion에 Label Encoding 적용
le = LabelEncoder()
data['predicted_emotion'] = le.fit_transform(data['predicted_emotion'])

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=300)
tfidf_matrix = tfidf.fit_transform(data['movie_info_keyword'])
tfidf_feature_names = tfidf.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)
data = pd.concat([data.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)
data.drop(columns=['movie_info_keyword'], inplace=True)

# Train-Test Split
train, test = train_test_split(data, test_size=0.2, random_state=42)

# User-Item Matrix 생성
user_movie_matrix = train.pivot_table(index='critic_name', columns='movie_id', values='review_score_cleaned').fillna(0)

# 사용자 간 코사인 유사도 계산
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Content Data 정의
numeric_cols = data.select_dtypes(include=[np.number]).columns
content_data = data[numeric_cols].copy()
content_data['movie_id'] = data['movie_id']
content_data['critic_name'] = data['critic_name']
content_data['movie_title'] = data['movie_title']


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm

def hybrid_recommend(user_name, user_movie_matrix, user_similarity_df, content_data, n_recommendations=5, top_n_users=300, user_weight=0.7):
    if user_name not in user_movie_matrix.index:
        raise ValueError(f"User '{user_name}' not found in user_movie_matrix.")

    # 사용자 기반 협업 필터링 추천 점수
    similar_users = user_similarity_df[user_name].sort_values(ascending=False).iloc[:top_n_users]
    similar_users = similar_users.reindex(user_movie_matrix.index).fillna(0)
    user_based_scores = user_movie_matrix.T.dot(similar_users).div(similar_users.sum())

    # Content-based 추천
    user_features = content_data[content_data['critic_name'] == user_name].drop(columns=['critic_name', 'movie_id', 'movie_title']).mean(axis=0).values.reshape(1, -1)
    content_scores = cosine_similarity(content_data.drop(columns=['critic_name', 'movie_id', 'movie_title']), user_features).ravel()
    content_recommendations = pd.Series(content_scores, index=content_data['movie_id'])

    # Hybrid 추천 (User-Based + Content-Based 합산)
    hybrid_scores = user_weight * user_based_scores.add(content_recommendations, fill_value=0)

    # 이미 본 영화 제외
    seen_movies = user_movie_matrix.loc[user_name]
    hybrid_scores = hybrid_scores[~hybrid_scores.index.isin(seen_movies[seen_movies > 0].index)]

    # 상위 추천 영화 반환
    hybrid_scores = hybrid_scores.reset_index()
    hybrid_scores.columns = ['movie_id', 'hybrid_score']
    recommendations = hybrid_scores.drop_duplicates(subset=['movie_id'], keep='first').sort_values(by='hybrid_score', ascending=False).head(n_recommendations)
    return recommendations


def evaluate_hybrid(test_data, user_movie_matrix, user_similarity_df, content_data, threshold=0.1, n_recommendations=100, top_n_users=500, user_weight=0.8):
    all_true = []
    all_predicted = []

    for user in tqdm(test_data['critic_name'].unique()):
        if user not in user_movie_matrix.index:
            continue

        # 테스트 데이터에서 실제 평점 추출
        user_actual = test_data[test_data['critic_name'] == user][['movie_id', 'review_score_cleaned']].set_index('movie_id')
        user_actual = user_actual[~user_actual.index.duplicated(keep='first')]

        # Hybrid 추천 점수 계산
        recommendations = hybrid_recommend(user, user_movie_matrix, user_similarity_df, content_data,
                                           n_recommendations=n_recommendations, top_n_users=top_n_users, user_weight=user_weight)

        # 이미 본 영화 제외
        recommendations = recommendations[~recommendations['movie_id'].isin(user_actual.index)]

        # 예측된 평점 추출
        predicted_ratings = recommendations.set_index('movie_id')['hybrid_score']
        predicted_ratings = predicted_ratings.reindex(user_actual.index).fillna(0)

        all_true.extend(user_actual['review_score_cleaned'])
        all_predicted.extend(predicted_ratings)

    # 평가 지표 계산
    rmse = np.sqrt(mean_squared_error(all_true, all_predicted))
    mae = mean_absolute_error(all_true, all_predicted)
    precision = precision_score(np.array(all_true) > threshold, np.array(all_predicted) > threshold, average='macro', zero_division=1)
    recall = recall_score(np.array(all_true) > threshold, np.array(all_predicted) > threshold, average='macro', zero_division=1)
    f1 = f1_score(np.array(all_true) > threshold, np.array(all_predicted) > threshold, average='macro', zero_division=1)

    return {'RMSE': rmse, 'MAE': mae, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}


# === 하이퍼파라미터 설정 ===
threshold = 0.1  # 기준 평점
n_recommendations = 100  # 추천 영화 수
top_n_users = 500  # 유사도가 높은 사용자 수
user_weight = 0.8  # 사용자 기반 필터링 가중치

In [7]:

# === 평가 실행 ===
metrics = evaluate_hybrid(test, user_movie_matrix, user_similarity_df, content_data,
                          threshold=threshold, n_recommendations=n_recommendations,
                          top_n_users=top_n_users, user_weight=user_weight)

print("Hybrid Recommendation System Metrics:")
print(metrics)

100%|██████████| 4395/4395 [1:12:07<00:00,  1.02it/s]


Hybrid Recommendation System Metrics:
{'RMSE': 0.6616046249557602, 'MAE': 0.625468156662476, 'Precision': 0.5065058180601509, 'Recall': 0.5, 'F1 Score': 0.01284450805534148}


In [6]:
metrics = evaluate_hybrid(test, user_movie_matrix, user_similarity_df, content_data, threshold=0.05, n_recommendations=300, top_n_users=300)
print("Hybrid Recommendation System Metrics:")
print(metrics)

100%|██████████| 4395/4395 [2:16:08<00:00,  1.86s/it]


Hybrid Recommendation System Metrics:
{'RMSE': 0.6616046249557602, 'MAE': 0.625468156662476, 'Precision': 0.5051042789694784, 'Recall': 0.5, 'F1 Score': 0.010105396414166691}


In [6]:

# === 사용자 추천 테스트 ===
user_name = "Ben McEachen"  # 추천을 받을 사용자 이름
n_recommendations = 5  # 추천할 영화 수
recommendations = hybrid_recommend(user_name, user_movie_matrix, user_similarity_df, content_data,
                                    n_recommendations=n_recommendations, top_n_users=top_n_users, user_weight=user_weight)

# 영화 제목 추가
recommendations = recommendations.merge(data[['movie_id', 'movie_title']], on='movie_id', how='left')

# 영화 제목 중복 제거
recommendations = recommendations.drop_duplicates(subset=['movie_title'])

# 상위 N개 추천
recommendations = recommendations.sort_values(by='hybrid_score', ascending=False).head(n_recommendations)
# 결과 출력
print(f"\nTop {n_recommendations} Hybrid Recommendations for {user_name}:")
print(recommendations[['movie_title', 'hybrid_score']])



Top 5 Hybrid Recommendations for Ben McEachen:
               movie_title  hybrid_score
0                Inception      1.040218
271              Star Trek      1.018579
541     The Social Network      1.016070
777                 Avatar      1.011679
1011  Inglourious Basterds      1.011361
